<center><img src="../Picture/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2021, Shenzhen Yahboom Tech  
@Author: ZiDan  
@Date: 2021-08-26    
@LastEditors: ZiDan    
@LastEditTime: 2021-08-26 

### mport-related driver library

In [3]:
import time
import Adafruit_SSD1306
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from jetbot.utils.utils import get_ip_address
import subprocess
from Battery_Vol_Lib import BatteryLevel

### Initialize SSD1306 

In [4]:
# 128x32 display and hardware I2C:
disp = Adafruit_SSD1306.SSD1306_128_32(rst=None, i2c_bus=0, gpio=1)# Set GPIO hack to 1 to avoid platform detection
# Initialize the library.
disp.begin()
# Clear display
disp.clear()
disp.display()
# Create a blank image for the drawing
# Make sure to create an image with the mode '1', i.e. 1-bit color
width = disp.width
height = disp.height
image = Image.new('1', (width, height))
# Gets the drawing object to draw on the image
draw = ImageDraw.Draw(image)
# Draw a black filled box to clear the image
draw.rectangle((0,0,width,height), outline=0, fill=0)
# Draw some shapes
# First, define some constants to make it easy to resize the shape
padding = -2
top = padding
bottom = height-padding
# Move left to right to track the current x position of the drawing.
x = 0
# Load default font
font = ImageFont.load_default()
# Create battery level query object
BatteryLevel = BatteryLevel()

OSError: [Errno 121] Remote I/O error

### Every second cycle refresh the information displayed on the OLED screen
Here, on the basis of the OLED routine, the captured battery information is displayed on the screen in the lower right corner of the OLED screen. 

Among them: 

B: H represents high battery 

B: M represents medium battery 

B: L represents low battery 

B: E represents empty battery

In [ ]:
while True:
    # Draw a black filled box to clear the image.
    draw.rectangle((0,0,width,height), outline=0, fill=0)

    # ou can get the shell script for system monitoring from this link : https://unix.stackexchange.com/questions/119126/command-to-display-memory-usage-disk-usage-and-cpu-load
    cmd = "top -bn1 | grep load | awk '{printf \"CPU Load: %.2f\", $(NF-2)}'"
    CPU = subprocess.check_output(cmd, shell = True )
    cmd = "free -m | awk 'NR==2{printf \"Mem:%s/%sM %.2f%%\", $3,$2,$3*100/$2 }'"
    MemUsage = subprocess.check_output(cmd, shell = True )
    cmd = "df -h | awk '$NF==\"/\"{printf \"Disk:%d/%dGB %s\", $3,$2,$5}'"
    Disk = subprocess.check_output(cmd, shell = True )

    draw.text((x, top),       "eth0:" + str(get_ip_address('eth0')),  font=font, fill=255)
    draw.text((x, top+8),     "wlan0:" + str(get_ip_address('wlan0')), font=font, fill=255)
    draw.text((x, top + 16), str(MemUsage.decode('utf-8')), font=font, fill=255)
    
    #Get the power by calling the battery power driver
    temp = BatteryLevel.Update()
    if temp == 'Battery_High':
        draw.text((x, top+25),    str(Disk.decode('utf-8')) + "  B:H",  font=font, fill=255)
        print('Battery_High')
    elif temp == 'Battery_Medium':
        draw.text((x, top+25),    str(Disk.decode('utf-8')) + "  B:M",  font=font, fill=255)
        print('Battery_Medium')
    elif temp =='Battery_Low':
        draw.text((x, top+25),    str(Disk.decode('utf-8')) + "  B:L",  font=font, fill=255)
        print('Battery_Low')
    elif temp == 'Battery_Empty':
        draw.text((x, top+25),    str(Disk.decode('utf-8')) + "  B:E",  font=font, fill=255)
        print('Battery_Empty')
    # draw.text((x, top+25),    str(Disk.decode('utf-8')),  font=font, fill=255)

    # Display image
    disp.image(image)
    disp.display()
    time.sleep(1)